# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,naze,28.3667,129.4833,79.72,73,1,11.65,JP,1730687005
1,1,hajiganj,23.2511,90.8592,76.12,70,0,5.12,BD,1730687007
2,2,invercargill,-46.4000,168.3500,63.01,61,100,11.01,NZ,1730687008
3,3,mangrol,21.1167,70.1167,78.69,52,2,6.51,IN,1730687009
4,4,constantia,44.1833,28.6500,32.68,49,3,11.50,RO,1730686779


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City")

# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_city = city_data_df[(
    city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 15)
    & (city_data_df["Cloudiness"] <= 80 ) & (city_data_df["Wind Speed"] <= 9.5)]

# Drop any rows with null values
ideal_weather_city.dropna()

# Display sample data
ideal_weather_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,31,lobujya,27.9500,86.8167,17.91,89,75,0.96,NP,1730687048
95,95,iqaluit,63.7506,-68.5145,17.33,78,75,2.30,CA,1730686858
172,172,yemetsk,63.4721,41.7941,18.50,97,65,4.00,RU,1730687226
188,188,labrador city,52.9463,-66.9114,26.19,73,75,8.05,CA,1730687248
208,208,kirensk,57.7853,108.1119,15.21,91,64,1.21,RU,1730687275
266,266,krasnyy chikoy,50.3643,108.7512,19.24,51,12,3.65,RU,1730687352
315,315,ust'-ilimsk,58.0006,102.6619,23.54,70,66,4.34,RU,1730687418
345,345,chibougamau,49.9168,-74.3659,26.22,100,0,5.75,CA,1730687458
463,463,banatski karlovac,45.0472,21.0161,26.35,34,3,1.99,RS,1730687616
520,520,kotlas,61.2611,46.6556,24.55,84,55,5.32,RU,1730687693


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_city[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
31,lobujya,NP,27.9500,86.8167,89,
95,iqaluit,CA,63.7506,-68.5145,78,
172,yemetsk,RU,63.4721,41.7941,97,
188,labrador city,CA,52.9463,-66.9114,73,
208,kirensk,RU,57.7853,108.1119,91,
266,krasnyy chikoy,RU,50.3643,108.7512,51,
315,ust'-ilimsk,RU,58.0006,102.6619,70,
345,chibougamau,CA,49.9168,-74.3659,100,
463,banatski karlovac,RS,45.0472,21.0161,34,
520,kotlas,RU,61.2611,46.6556,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lobujya - nearest hotel: Hôtel Mother Earth
iqaluit - nearest hotel: Frobisher Inn
yemetsk - nearest hotel: No hotel found
labrador city - nearest hotel: Carol Inn
kirensk - nearest hotel: Ивушка
krasnyy chikoy - nearest hotel: Гостиница ЗАО "Слюдянка"
ust'-ilimsk - nearest hotel: Усть-Илимск
chibougamau - nearest hotel: No hotel found
banatski karlovac - nearest hotel: No hotel found
kotlas - nearest hotel: Советская
tosontsengel - nearest hotel: Алтай


,City,Country,Lat,Lng,Humidity,Hotel Name
31,lobujya,NP,27.9500,86.8167,89,Hôtel Mother Earth
95,iqaluit,CA,63.7506,-68.5145,78,Frobisher Inn
172,yemetsk,RU,63.4721,41.7941,97,No hotel found
188,labrador city,CA,52.9463,-66.9114,73,Carol Inn
208,kirensk,RU,57.7853,108.1119,91,Ивушка
266,krasnyy chikoy,RU,50.3643,108.7512,51,"Гостиница ЗАО ""Слюдянка"""
315,ust'-ilimsk,RU,58.0006,102.6619,70,Усть-Илимск
345,chibougamau,CA,49.9168,-74.3659,100,No hotel found
463,banatski karlovac,RS,45.0472,21.0161,34,No hotel found
520,kotlas,RU,61.2611,46.6556,84,Советская


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
ideal_city_map = hotel_df.hvplot.points("Lng", 
                                          "Lat", 
                                          geo = True, 
                                          tiles = "OSM", 
                                          frame_width = 800, 
                                          frame_height = 600, 
                                          size = "Humidity", 
                                          scale = 1, 
                                          color = "City", 
                                          hover_cols = ["City", "Hotel Name", "Country"])

# Display the map
ideal_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)